# 何も確認せずにまず以下を実行してみましょう！！

In [3]:
import cv2
import numpy as np

# カメラからの入力を開始
cap = cv2.VideoCapture(0)
# 動画書き出し用のオブジェクトを生成
# 'm','p','4','v'は、動画コーデック(＝動画の圧縮形式)
fmt = cv2.VideoWriter_fourcc('m','p','4','v')
# 動画の再生速度(fps)
fps = 20.0
# 解像度
size = (640, 360)
# 動画ファイルとして保存するために VideoWriter 型のオブジェクトを生成
# cv2.VideoWriter(ファイル名、コーデック、再生速度、大きさ)
writer = cv2.VideoWriter('test.m4v', fmt, fps, size)

while True:
    # 成功/失敗、読み込んだ画像
    _, frame = cap.read() # 動画を入力
    # 画像を縮小(640, 360)
    frame = cv2.resize(frame, size)
    # 画像を出力 動画として書き出す1コマ分の静止画（frame）を書き込む
    writer.write(frame)
    # ウィンドウ上にも表示
    cv2.imshow('frame', frame)
    # Enterキーが押されたらループを抜ける
    if cv2.waitKey(1) == 13: break
    
writer.release()
cap.release()
cv2.destroyAllWindows() # ウィンドウを破棄


In [6]:
# fishvideo_extract_diff.py
import cv2, os

img_last = None # 前回の画像
no = 0 # 画像の枚数
save_dir = "./exfish" # 保存ディレクトリ名

# exfishフォルダが作成済みの場合エラーになってしまうので注意
os.mkdir(save_dir) # ディレクトリを作成

# 動画ファイルから入力を開始
cap = cv2.VideoCapture("fish.mp4")
while True:
    # 画像を取得
    is_ok, frame = cap.read()
    # is_okにはread()の結果が入っていて、Trueなら正常、Falseなら異常
    # not(is_ok==True):break という意味
    if not is_ok: break
    frame = cv2.resize(frame, (640, 360))
    # 白黒画像に変換
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (15, 15), 0)
    img_b = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)[1]
    # 差分を確認する
    if not img_last is None:
        #absdiffは２枚の画像の差分の絶対値を取得する      
        # 前のコマと今のコマを比較する
        frame_diff = cv2.absdiff(img_last, img_b)
        cnts = cv2.findContours(frame_diff, 
            cv2.RETR_EXTERNAL,
            cv2.CHAIN_APPROX_SIMPLE)[0]
        # 差分があった領域をファイルに出力
        for pt in cnts:
            x, y, w, h = cv2.boundingRect(pt)
            if w < 100 or w > 500: continue # ノイズを除去
            # 抽出した領域を画像として保存
            imgex = frame[y:y+h, x:x+w]
            # ファイル名の相対パス
            outfile = save_dir + "/" + str(no) + ".jpg"
            # 　　　　　　　　　　↓切り出して変化があった部分
            cv2.imwrite(outfile, imgex)
            no += 1
    img_last = img_b
cap.release()
print("ok")


FileExistsError: [WinError 183] 既に存在するファイルを作成することはできません。: './exfish'

## glob について調べよう

https://note.nkmk.me/python-glob-usage/

In [1]:
# fish_train.py
import cv2
import os, glob, pickle
from sklearn.model_selection import train_test_split
from sklearn import datasets, metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# 画像の学習サイズやパスを指定
image_size = (64, 32)

# 実行中のフォルダのフルパス
# path = os.path.dirname(os.path.abspath(__file__))
# \→/に変更して自分の実行中のフォルダのフルパスをpathに代入する
# 例）
path = 'C:\\Users\\k_kurosaki\\AIFW2\\0601'

path_fish = path + '/fish'
path_nofish = path + '/nofish'
x = [] # 画像データ
y = [] # ラベルデータ

# 画像データを読み込んで配列に追加
def read_dir(path, label):
    # glob.glob(引数)：引数のパスの中から、条件に合致するファイルを全部持ってくる
    # *(ワイルドカード)：1文字以上の文字列(どんな文字でもok)
    files = glob.glob(path + "/*.jpg")
    for f in files:
        img = cv2.imread(f)
        img = cv2.resize(img, image_size)
        img_data = img.reshape(-1, ) # 一次元に展開
        # x：特徴量、y：正解データ
        x.append(img_data)
        y.append(label)

# 画像データを読み込む
read_dir(path_nofish, 0)
read_dir(path_fish, 1)

# データを学習用とテスト用に分割する
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

# データを学習
clf = RandomForestClassifier()
clf.fit(x_train, y_train)

# 精度の確認
y_pred = clf.predict(x_test)
print(accuracy_score(y_test, y_pred))

# データを保存
with open("fish.pkl", "wb") as fp:
  pickle.dump(clf, fp)

0.9411764705882353
